# Player evaluation

Here we compare players based on their adjusted points. Also, we calculate some useful metrics using the adjusted points to further evaluate player performance in FPL.

In [1]:
# recent gameweek
gameweek = 21

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
filepath = '../data/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0, encoding='latin-1')

# fetch team data
filepath = '../data/fbref/team_stats_week' + str(gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0, encoding='latin-1')

# fetch player stats data
filepath = '../data/fbref/player_stats_week' + str(gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1, encoding='latin-1')

## Value and value points

Here we calculate two interesting metrics: 'value' and 'value points'. 

Value is simply adjusted points per game divided by the cost of the player. Essentially, this measures how many 'points per pound(/euro/whatever)' has the player gained on an average game week. 

The value points is calculated as the geometric mean of a players adjusted points per game and value. Essentially, in FPL we want to gain the maximum possible amount of points. However, we are restrained by a limited budget, so that in practice we cannot just pick all the best players. Then, we would like to have a squad of players that give the most points given our budget. Thus, we would like to pick players that have high value ('bang for buck'). However, often the players with highest value are very cheap players who somewhat overperform relative to their price. Picking a squad full of these players might leave a part of our budget unused. Calculating the geometric mean of adjusted points and value allows us to weigh both aspects equally, i.e. we want players that both gain a lot of points but also have good value. For example, this metric values equally two players, where one has twice the points per game of the other who in turn has twice the value of the other.

In [4]:
# value = expected points / cost
df['value'] = df['adjusted points per game'] / (df['now_cost'] / 10.0)
# geometric mean of 'adjusted points per game' and 'value'
df['valuePoints metric'] = np.sqrt(df['adjusted points per game'] * df['value'])

In [5]:
# assign proper team names for each player
team_names = np.sort(teamStats['Squad'])
df['team_name'] = team_names[df['team']-1]

In [6]:
# save data
filepath = '../data/data_week' + str(gameweek) + str('.csv')
df.to_csv(filepath)

In [7]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 3

## Player evaluation

Below we compile lists for each position sorting players based on a given metric.

In [8]:
#metric = 'valuePoints metric'
metric = 'adjusted points per game'
#metric = 'form 5'

numberToShow = 40

print('GOALKEEPERS')
display(df[goalkeepers & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))
print('DEFENDERS')
display(df[defenders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))
print('MIDFIELDERS')
display(df[midfielders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))
print('FORWARDS')
display(df[forwards & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))

GOALKEEPERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
411,PatrÃÂ­cio,Wolves,52,20.810811,77,90.619521,3.7,4.354445,3.982564,1.909550
388,Fabianski,West Ham,49,9.019608,46,36.145347,5.1,4.007419,5.475934,1.810367
131,Guaita,Crystal Palace,51,18.809524,79,74.832547,4.2,3.978439,4.438857,1.761682
235,de Gea,Manchester Utd,54,21.000000,63,82.949296,3.0,3.949966,3.302529,1.699795
340,Lloris,Tottenham,53,6.969697,23,27.238502,3.3,3.908133,NaN,1.697584
47,Ryan,Brighton,48,20.769231,81,80.720757,3.9,3.886555,3.972144,1.773962
14,Leno,Arsenal,50,20.833333,75,78.970920,3.6,3.790604,4.452262,1.695210
471,Henderson,Sheffield Utd,49,20.250000,81,76.554539,4.0,3.780471,3.877531,1.707843
494,Ramsdale,Bournemouth,47,20.810811,77,77.926756,3.7,3.744532,3.393762,1.727224


DEFENDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
182,Alexander-Arnold,Liverpool,75,20.000000,112,115.112599,5.6,5.755630,6.628820,2.101659
181,Robertson,Liverpool,70,20.000000,98,104.520169,4.9,5.226008,6.531925,1.975246
297,Lundstram,Sheffield Utd,51,20.000000,98,97.381847,4.9,4.869092,3.326310,2.156070
401,Doherty,Wolves,61,19.210526,73,85.111198,3.8,4.430446,4.732070,1.793835
185,Matip,Liverpool,52,6.956522,32,30.289225,4.6,4.354076,NaN,1.909389
103,Alonso,Chelsea,60,7.966102,47,34.580472,5.9,4.340953,6.851449,1.772187
183,van Dijk,Liverpool,64,20.000000,92,84.781523,4.6,4.239076,4.510940,1.675642
105,Azpilicueta,Chelsea,58,19.310345,56,77.792391,2.9,4.028535,7.180607,1.672758
405,Boly,Wolves,48,8.095238,34,32.144863,4.2,3.970836,NaN,1.812430


MIDFIELDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
191,Salah,Liverpool,123,16.901408,120,116.027963,7.1,6.864988,8.807730,1.957434
192,ManÃÂ©,Liverpool,123,18.918919,140,126.424410,7.4,6.682433,7.790412,1.905382
215,De Bruyne,Manchester City,106,20.142857,141,124.898446,7.0,6.200632,5.054441,1.904509
214,Sterling,Manchester City,118,20.000000,110,113.468851,5.5,5.673443,3.977805,1.651603
239,Martial,Manchester Utd,79,15.000000,81,72.774554,5.4,4.851637,4.097451,1.726135
171,Maddison,Leicester City,77,19.807692,103,94.338590,5.2,4.762725,3.291950,1.716367
217,Mahrez,Manchester City,84,17.115385,89,78.451347,5.2,4.583674,5.144131,1.581518
431,Pulisic,Chelsea,70,15.909091,70,72.466475,4.4,4.555036,2.665095,1.721642
344,Alli,Tottenham,88,13.888889,75,62.073649,5.4,4.469303,3.606063,1.506602


FORWARDS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
166,Vardy,Leicester City,101,18.947368,144,117.115538,7.6,6.181098,4.504531,1.944934
233,Rashford,Manchester Utd,91,21.034483,122,120.715538,5.8,5.738935,3.850116,1.902439
460,Abraham,Chelsea,78,19.827586,115,102.092430,5.8,5.149010,3.058322,1.843641
11,Aubameyang,Arsenal,108,20.847458,123,106.569323,5.9,5.111862,3.435784,1.555490
409,JimÃÂ©nez,Wolves,75,21.200000,106,105.838645,5.0,4.992389,4.543627,1.822963
210,AgÃÂ¼ero,Manchester City,117,14.909091,82,73.869323,5.5,4.954650,1.768531,1.448505
167,Iheanacho,Leicester City,57,5.000000,30,24.546215,6.0,4.909243,3.054331,2.056256
313,Ings,Southampton,67,21.071429,118,102.123108,5.6,4.846520,6.057166,1.872374
278,Pukki,Norwich City,65,20.833333,100,97.769323,4.8,4.692927,3.457212,1.840718


Below we compile a list sorting players based on a given metric (irrespective of position).

In [9]:
#metric = 'adjusted points per game'
metric = 'form 5'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
191,Salah,Liverpool,123,16.901408,120,116.027963,7.1,6.864988,8.807730,1.957434
192,ManÃÂ©,Liverpool,123,18.918919,140,126.424410,7.4,6.682433,7.790412,1.905382
105,Azpilicueta,Chelsea,58,19.310345,56,77.792391,2.9,4.028535,7.180607,1.672758
103,Alonso,Chelsea,60,7.966102,47,34.580472,5.9,4.340953,6.851449,1.772187
265,PÃÂ©rez,Leicester City,60,18.043478,83,70.202134,4.6,3.890721,6.797775,1.588380
182,Alexander-Arnold,Liverpool,75,20.000000,112,115.112599,5.6,5.755630,6.628820,2.101659
181,Robertson,Liverpool,70,20.000000,98,104.520169,4.9,5.226008,6.531925,1.975246
283,BuendÃÂ­a,Norwich City,60,20.909091,69,79.923622,3.3,3.822434,6.152588,1.560502
313,Ings,Southampton,67,21.071429,118,102.123108,5.6,4.846520,6.057166,1.872374
